In [1]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path
import shutil
import datetime as dt
from pprint import pprint

import pandas as pd

from wattile.entry_point import init_logging
from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import prep_for_rnn
from wattile.models import ModelFactory


PROJECT_DIRECTORY = Path().resolve().parents[1]


PROJECT_DIRECTORY = /Users/jsmith2/Code/IC/Wattile/wattile


In [2]:
"""
This demo runs predictions on a pre-trained alpha model in ./tests/fixtures/alfa_exp_dir.
"""
exp_dir = PROJECT_DIRECTORY / "tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1"
with open(exp_dir / "configs.json", "r") as f:
    configs = json.load(f)

print(configs)

{'data_path': '../../data/STM Campus', 'feature_extraction': 'no', 'scenario_filter': 'no', 'scenario_wrapper': 'no', 'metric_relevancy': 'none', 'metric_wrapper': 'Interval Score', 'num_cores_wrapper': 1, 'mi_rand_frac': 1.0, 'threshold_basis': 'percentile', 'threshold_relevancy': 0.0, 'threshold_redundancy': 1.0, 'target_clean_size_weeks': 1, 'code_dir': '/projects/intelcamp/repos/WattChoice', 'results_dir': '/projects/intelcamp/repos/WattChoice/results/FTLB', 'generate_plot': True, 'generate_plot_inputdata': True, 'data_input': {'data_dir': '/projects/intelcamp/repos/WattChoice/../../data/STM Campus', 'data_config': 'FTLB Config.json', 'start_time': '2020-07-01T00:00:00-07:00', 'end_time': '2023-04-01T00:00:00-07:00', 'predictor_columns': [], 'target_var': 'FTLB CHW Meter CHW Energy Rate'}, 'data_output': {'exp_dir': '/projects/intelcamp/repos/WattChoice/results/FTLB/FTLB_FTLBCHWMeterCHWEnergyRate_r1', 'plot_comparison': True, 'plot_comparison_portion_start': 0.0, 'plot_comparison_p

In [3]:
"""
Init the model with it's configs. Be sure to switch it's use case to prediction.
"""

configs["data_output"]["exp_dir"] = exp_dir
configs["learning_algorithm"]["use_case"] = "prediction"
init_logging(local_results_dir=configs["data_output"]["exp_dir"])
model = ModelFactory.create_model(configs)

model

Logging to: /Users/jsmith2/Code/IC/Wattile/tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1/output.out, PID: 13230


In [4]:
"""
If we want to get predictions for only one row, we have to find the time range needed to predict the nominal time.

we can use `get_input_window_for_output_time` to figure this out.
"""
nominal_time = pd.Timestamp(year=2023, month=1, day=1, tz=dt.timezone.utc)

start_time, end_time = model.get_input_window_for_output_time(nominal_time)
freq = model.configs["data_processing"]["resample"]["bin_interval"]
inclusive = model.configs["data_processing"]["resample"]["bin_label"]

predictors = model.configs["data_input"]["predictor_columns"]
target_var = model.configs["data_input"]["target_var"]

print(f"""
    To use this model to predict {target_var} for {nominal_time}, we will need to feed it data:
        - starting at {start_time}
        - ending at {end_time}
        - a rough frequency of {freq}
        - predictors named {predictors}
""")


    To use this model to predict FTLB CHW Meter CHW Energy Rate for 2023-01-01 00:00:00+00:00, we will need to feed it data:
        - starting at 2022-12-31 18:00:00+00:00
        - ending at 2023-01-01 00:15:00+00:00
        - a rough frequency of 15min
        - predictors named []



In [5]:
"""
Great, now lets make some dummy data that matches those specs.
"""

# data = pd.DataFrame(index=pd.date_range(start_time, end_time, freq=freq, inclusive=inclusive))
# for predictor in predictors:
#     data[predictor] = data.index.hour * 100 + data.index.minute

# # even though we are predicting, we must fill the target column with dummy data. It's not used though.
# data[target_var] = data.index.hour * 100 + data.index.minute


# print(data.shape)

data = pd.read_csv(f'{PROJECT_DIRECTORY}/tests/data/test_predictor_data.csv')
data['ts'] = pd.to_datetime(data['ts'])
data.set_index('ts', inplace=True)
cols_list = data.columns.to_list()
print(f'cols shape: {data.shape}')
print(f'cols len: {len(data.columns)}')
print(f'cols: {cols_list}\n')

with open(f'{PROJECT_DIRECTORY}/tests/fixtures/FTLB_FTLBCHWMeterCHWEnergyRate_r1/predictors_target_config.json') as f:
    pred_data = json.load(f)

pred_cols = [pred['column'] for pred in pred_data['predictors']]
print(f'pred cols len: {len(pred_cols)}')
print(f'pred cols: {pred_cols}\n')

the_diff = list(set(cols_list) - set(pred_cols))
print(f'diff between lists: {the_diff}')

print(data.index)

cols shape: (3911, 14)
cols len: 14
cols: ['FTLB CHW Meter CHWST', 'FTLB HW Meter HWST', 'SRRL BMS Dew Point Temperature', 'SRRL BMS Diffuse Horizontal Irradiance', 'SRRL BMS Direct Normal Irradiance', 'SRRL BMS Dry Bulb Temperature', 'SRRL BMS Global Horizontal Irradiance', 'SRRL BMS Rainfall', 'SRRL BMS Relative Humidity', 'SRRL BMS Snow Depth', 'SRRL BMS Wet Bulb Temperature', "SRRL BMS Wind Direction at 19'", "SRRL BMS Wind Speed at 19'", 'FTLB CHW Meter CHW Energy Rate']

pred cols len: 13
pred cols: ['FTLB CHW Meter CHWST', 'FTLB HW Meter HWST', 'SRRL BMS Dew Point Temperature', 'SRRL BMS Diffuse Horizontal Irradiance', 'SRRL BMS Direct Normal Irradiance', 'SRRL BMS Dry Bulb Temperature', 'SRRL BMS Global Horizontal Irradiance', 'SRRL BMS Rainfall', 'SRRL BMS Relative Humidity', 'SRRL BMS Snow Depth', 'SRRL BMS Wet Bulb Temperature', "SRRL BMS Wind Direction at 19'", "SRRL BMS Wind Speed at 19'"]

diff between lists: ['FTLB CHW Meter CHW Energy Rate']
DatetimeIndex(['2022-12-31 1

In [14]:
"""
Let's prep those for the model.
"""
train_df, val_df = prep_for_rnn(configs, data)


# print(f'{val_df.columns}')

prep_for_rnn
init cols: 14
Index(['FTLB CHW Meter CHWST', 'FTLB HW Meter HWST',
       'SRRL BMS Dew Point Temperature',
       'SRRL BMS Diffuse Horizontal Irradiance',
       'SRRL BMS Direct Normal Irradiance', 'SRRL BMS Dry Bulb Temperature',
       'SRRL BMS Global Horizontal Irradiance', 'SRRL BMS Rainfall',
       'SRRL BMS Relative Humidity', 'SRRL BMS Snow Depth',
       'SRRL BMS Wet Bulb Temperature', 'SRRL BMS Wind Direction at 19'',
       'SRRL BMS Wind Speed at 19'', 'FTLB CHW Meter CHW Energy Rate'],
      dtype='object')
correct_predictor_columns time cols added
cols: 14
correct_timestamps time cols added
cols: 14
add_processed_time_columns time cols added
cols: 80
processed time cols added
cols: 5926
Index(['FTLB CHW Meter CHWST_min_lag24', 'FTLB HW Meter HWST_min_lag24',
       'SRRL BMS Dew Point Temperature_min_lag24',
       'SRRL BMS Diffuse Horizontal Irradiance_min_lag24',
       'SRRL BMS Direct Normal Irradiance_min_lag24',
       'SRRL BMS Dry Bulb Temperatu

In [7]:
"""
One line in, one prediction (with many quantiles) out.
"""
results = model.predict(val_df)

results

<xarray.DataArray (timestamp: 96, quantile: 9, horizon: 1)>
array([[[247.74109],
        [296.90662],
        [330.71387],
        [356.90125],
        [381.69263],
        [407.37256],
        [435.80603],
        [469.73743],
        [525.2406 ]],

       [[249.11926],
        [298.40686],
        [332.29492],
        [358.56946],
        [383.45984],
        [409.2555 ],
        [437.82886],
        [471.93506],
        [527.6582 ]],

...

       [[214.11938],
        [260.30115],
        [292.14465],
        [316.20105],
        [338.57654],
        [361.4392 ],
        [386.46118],
        [416.12695],
        [466.25696]],

       [[213.7102 ],
        [259.8557 ],
        [291.6754 ],
        [315.7057 ],
        [338.05212],
        [360.8806 ],
        [385.86084],
        [415.4746 ],
        [465.53918]]], dtype=float32)
Coordinates:
  * timestamp  (timestamp) object 2023-01-01T00:00:00-07:00 ... 2023-01-01T23...
  * quantile   (quantile) float64 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9
  * horizon    (horizon) timedelta64[ns] 00:00:00